In [199]:
import sys # to access the system
import cv2
from bs4 import BeautifulSoup
import bs4
import time
import os
# import filecmp
# cmp = filecmp.dircmp(DBROOT+r"\task_20230718085420_20230718085435_3.mp4_dataset_2024_02_28_10_02_13_cvat for video 1.1 (1)\images", DBROOT+r"\task_20230718085420_20230718085435_3.mp4_dataset_2024_02_28_10_02_13_cvat for video 1.1\images")
# cmp.report()

def path2img(img_num, dbpath):
    img_pad = str(img_num).zfill(6)
    return f"{dbpath}\\frame_{img_pad}.PNG"

def load_annotations(annotation_path: str):
    r"""
    annotation_path: str = "D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_02_05_18_52_05-cvat for video 1.1"
    """
    path=f"{annotation_path}\\annotations.xml"
    with open(path, 'r') as f:
        data = f.read()
    Bs_data = BeautifulSoup(data, "xml")
    metadata = Bs_data.find('meta').find("task").find("segments")
    # Get the annotations frame count
    start_frame = int(metadata.find("start").text)
    stop_frame = int(metadata.find("stop").text)
    # The annotations is within track property
    box_annotations = Bs_data.find_all('track')
    # Check if there are any annotations of fish, else return empty box annotations
    if box_annotations:
        # Only get box elements
        map_frame_to_index = {}
        all_boxes = box_annotations[0].find_all('box')
        for i in range(1, len(box_annotations)):
            all_boxes += box_annotations[i].find_all('box')
        for j, box in enumerate(all_boxes):
            #print(str(box["frame"]))
            if box["frame"] in map_frame_to_index:
                map_frame_to_index[box["frame"]] += [j]
            else:
                map_frame_to_index[box["frame"]] = [j]
        return all_boxes, map_frame_to_index, (start_frame, stop_frame)
    return "", {}, (start_frame, stop_frame)

def showVideo(samples_path: str, window: str, skip_empty: bool=False, window_name: str= "") -> bool:
    r"""
    samples_path = "D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_01_28_18_07_53-cvat for video 1.1"
    """
    # Draw boxes settings
    color = (0, 255, 0)  # Green
    thickness = 10
    # Load annotation boxes and their mapping
    boxes, map2indices, frames = load_annotations(samples_path)
    video_name = window_name if window_name else samples_path.split("\\")[-1]
    cv2.setWindowTitle(window, video_name)
    if skip_empty and not boxes:
        return True
    for i in range(*frames):
        img = cv2.imread(path2img(i, samples_path+r"\images"), cv2.IMREAD_ANYCOLOR)
        if str(i) in map2indices:
            for indicie in map2indices[str(i)]:
                box = boxes[indicie]
                # Get coordinates from your box data
                x = float(box['xtl'])
                y = float(box['ytl'])
                x2 = float(box['xbr'])
                y2 = float(box['ybr'])
                # Convert to integers
                x, y, x2, y2 = map(int, [x, y, x2, y2])
                # Draw rectangle
                cv2.rectangle(img, (x, y), (x2, y2), color, thickness)
        key = cv2.waitKey(2) & 0xFF
        if key == ord("q"):
            return True
        elif key == ord("a"):
            return False
        #samples_path.split("\\")[-1],
        cv2.imshow(window, img)
    return True



# img = cv2.imread("sheep.png", cv2.IMREAD_ANYCOLOR)
def loadLabelsYolo(label_filepath: str) -> list[tuple[int, float, float, float, float]]:
    r"""
    Load YOLO label data from a specified file.

    This function reads a label file containing label information in a specific format and 
    returns a list of tuples. Each tuple contains the following information:
    
    - label int: The integer identifier for the label.
    - x float: The x-coordinate of the label's bounding box.
    - y float: The y-coordinate of the label's bounding box.
    - width float: The width of the label's bounding box.
    - height float: The height of the label's bounding box.

    Args:
        label_filepath (str): The path to the label file.

    Returns:
        List[Tuple[int, float, float, float, float]]: A list of tuples where each tuple 
        corresponds to a label in the format (label int, x float, y float, width float, height float).
    """
    
    # Open the label file and read its contents
    with open(label_filepath, "r") as f:
        data = f.readlines()
    
    # Strip whitespace and split each line into components
    rstripped = [d.rstrip().split() for d in data]
    
    # Convert the split string data into a list of tuples with the appropriate types
    labels = [
        (int(l[0]), float(l[1]), float(l[2]), float(l[3]), float(l[4])) 
        for l in rstripped
    ]
    
    return labels

def labelToBox(label: tuple[float, int, int, int, int], window_size: tuple[int, int]=(1920,1080)) -> tuple[tuple[int, int], tuple[int, int]]:
    """
    Eg label = (0, 0.9653645833333333, 0.1361111111111111, 0.06822916666666666, 0.1574074074074074)
    Eg. output = (1788, 62), (1919, 232)
    """
    # 0.5, 0.5, 0.1, 0.1
    width = int(window_size[0] * label[3])
    height = int(window_size[1] * label[4])
    
    start_x = int(label[1] * window_size[0] - width/2)
    start_y = int(label[2] * window_size[1] - height/2)

    end_x = start_x + width
    end_y = start_y + height
    return (start_x,start_y),(end_x,end_y)

    


def readLabelYolo(path: str):
    with open(path, "r") as reader:
        data = reader.readlines()
    return [line.rstrip() for line in data]
    #print(data)

def labelpathToImagepath(label_path: str) -> str:
    labels_path, label_name = os.path.split(label_path)
    images_path = os.path.join(os.path.split(labels_path)[0], "images")
    image_path = os.path.join(images_path, label_name[:-3] + "PNG")
    return image_path

def displayImageNotated(label_path: str):
    detections = readLabelYolo(label_path)
    image_path = labelpathToImagepath(label_path)
    img = cv2.imread(image_path, cv2.IMREAD_ANYCOLOR)
    if len(detections) > 0:
        for detection in detections:
            split_det = detection.split()
            #if float(split_det[3]) > 0.25 and float(split_det[3]) < 0.80:
            start,stop = labelToBox((int(split_det[0]), float(split_det[1]),float(split_det[2]),float(split_det[3]),float(split_det[4])))
            cv2.rectangle(img, start, stop, color, thickness)
    
    cv2.imshow("window", img)
    key = cv2.waitKey(2)
    print(detections, directory)

def saveImageNotated(label_path: str, dst_path: str):
    detections = readLabelYolo(label_path)
    image_path = labelpathToImagepath(label_path)
    img = cv2.imread(image_path, cv2.IMREAD_ANYCOLOR)
    if len(detections) > 0:
        for detection in detections:
            split_det = detection.split()
            #if float(split_det[3]) > 0.25 and float(split_det[3]) < 0.80:
            start,stop = labelToBox((int(split_det[0]), float(split_det[1]),float(split_det[2]),float(split_det[3]),float(split_det[4])))
            cv2.rectangle(img, start, stop, color, thickness)
    cv2.imwrite(dst_path, img)
    #cv2.imshow("window", img)
    #key = cv2.waitKey(2)
    #print(detections, directory)


In [223]:
DBPATH = r"D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_01_28_18_07_53-cvat for video 1.1\images"
ANNOTATIONS= r"D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_01_28_18_07_53-cvat for video 1.1"
DBROOT= r"D:\Dataset\FishDetection\Annotation"
test_path=r"D:\Dataset\FishDetection2Yolo\valid\labels"
test_imgpath=r"D:\Dataset\FishDetection2Yolo\valid\images"

windowID = "Display"
color = (0, 255, 0)  # Green
thickness = 10

directories = os.listdir(test_path)
total_detections = {}
for directory in directories:
    label_path = os.path.join(test_path, directory)
    detections = readLabelYolo(label_path)
    detectionCount = len(detections)
    total_detections[str(detectionCount)] = total_detections.get(str(detectionCount),0) +1
    if len(detections) > 10:
        #print(label_path)
        #print(detections)
        print(label_path, os.path.join("anomalies",directory))
        saveImageNotated(label_path, os.path.join("anomalies",directory)[:-3]+"PNG")
        #print(label_path, os.path.join("anomalies",directory))
        
        if(False):
            split_det = detections[0].split()
            if float(split_det[3]) > 0.25 and float(split_det[3]) < 0.80:
                print(os.path.join(test_imgpath, directory[:-3]+"PNG"))
                img = cv2.imread(os.path.join(test_imgpath, directory[:-3]+"PNG"), cv2.IMREAD_ANYCOLOR)
                start,stop = labelToBox((int(split_det[0]), float(split_det[1]),float(split_det[2]),float(split_det[3]),float(split_det[4])))
                cv2.rectangle(img, start, stop, color, thickness)
    
                cv2.imshow("window", img)
                key = cv2.waitKey(10075)
                if key == ord("q"):
                    cv2.destroyAllWindows()
                break
                print(detections, directory)
    #childdir = os.path.join(DBROOT,directory)
print(total_detections)
#{'1': 17393, '0': 12071, '2': 1281, '3': 308, '4': 31, '5': 16} TRAIN
#{'2': 317, '1': 2738, '0': 1238, '3': 105} TEST
#{'0': 1742, '1': 2450, '2': 118, '8': 50, '10': 10, '13': 11, '11': 29, '12': 11, '9': 10, '7': 10, '6': 10} VALID



D:\Dataset\FishDetection2Yolo\valid\labels\frame_001607.txt anomalies\frame_001607.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001608.txt anomalies\frame_001608.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001609.txt anomalies\frame_001609.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001610.txt anomalies\frame_001610.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001611.txt anomalies\frame_001611.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001612.txt anomalies\frame_001612.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001613.txt anomalies\frame_001613.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001614.txt anomalies\frame_001614.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001615.txt anomalies\frame_001615.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001616.txt anomalies\frame_001616.txt
D:\Dataset\FishDetection2Yolo\valid\labels\frame_001617.txt anomalies\frame_001617.txt
D:\Dataset\FishDetection2Yolo\valid\labels\

In [177]:
img = cv2.imread(r"D:\Dataset\FishDetection2Yolo\valid\images\frame_001608.PNG", cv2.IMREAD_ANYCOLOR)
cv2.imshow("window", img)

In [221]:
saveImageNotated(r"D:\Dataset\FishDetection2Yolo\valid\labels\frame_001607.txt", "anomalies\frame_001607.png")

In [195]:
shutil.copyfile(r"D:\Dataset\FishDetection2Yolo\valid\images\frame_001608.PNG", "anomalies/test.PNG")

'anomalies/test.PNG'

In [179]:
cv2.destroyAllWindows()

In [11]:

directories = os.listdir(DBROOT)
pngs = 0
for directory in directories:
    childdir = os.path.join(DBROOT,directory)
    pngs+=len(os.listdir(os.path.join(childdir, "images")))
print(pngs)

80160


In [15]:
directories = os.listdir(DBROOT)
windowID = "Display"
cv2.namedWindow(windowID, cv2.WINDOW_AUTOSIZE) 
try:
    for i in range(0, len(directories), 3):
        print(os.path.join(DBROOT,directories[i]))
        continueNext = showVideo(os.path.join(DBROOT, directories[i]),windowID, skip_empty=True)
        if not continueNext:
            break

finally:       
    cv2.destroyAllWindows()


D:\Dataset\FishDetection\Annotation\task_20230716011520_20230716011528_3.mp4_dataset_2024_02_24_12_58_00_cvat for video 1.1


In [5]:
root = r"C:\Users\alf\Dataset\DeepFish\test"
directories = os.listdir(root)
label_files = [os.path.join(root,a) for a in directories if "txt" in a]

In [7]:
print(label_files[0].rstrip(".txt")+".jpg")
print(label_files[0])


C:\Users\alf\Dataset\DeepFish\test\7117_Caranx_sexfasciatus_juvenile_f000006.jpg
C:\Users\alf\Dataset\DeepFish\test\7117_Caranx_sexfasciatus_juvenile_f000006.txt


In [47]:
color = (0, 255, 0)  # Green
thickness = 10
cv2.setWindowTitle("window", "test")
# path=r"D:\Dataset\FishDetection\Annotation\task_20230716011538_20230716011547_3.mp4_dataset_2024_02_24_12_58_15_cvat for video 1.1\images\frame_000156.PNG"
# img = cv2.imread(label_files[0].rstrip(".txt")+".jpg", cv2.IMREAD_COLOR)
print(label_files[1].rstrip(".txt")+".jpg")
for label_file in label_files:
    img = cv2.imread(label_file.rstrip(".txt")+".jpg", cv2.IMREAD_COLOR)
    # img.shape gives x and y reversed order
    y,x,_ = img.shape
    labels = loadLabelsYolo(label_file)
    for label in labels:
        start, stop = labelToBox(label, (x,y))
        cv2.rectangle(img, start, stop, color, thickness)
    cv2.imshow("window", img)
    key = cv2.waitKey(75)
    if key == ord("q"):
        cv2.destroyAllWindows()
        break




C:\Users\alf\Dataset\DeepFish\test\7117_Caranx_sexfasciatus_juvenile_f000008.jpg


In [51]:
len(label_files)

909

In [ ]:
def showVideoByPath(samples_path: str, window: str, skip_empty: bool=False, window_name: str= "") -> bool:
    r"""
    samples_path = "D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_01_28_18_07_53-cvat for video 1.1"
    """
    # Draw boxes settings
    color = (0, 255, 0)  # Green
    thickness = 10
    # Load annotation boxes and their mapping
    boxes, map2indices, frames = load_annotations(samples_path)
    video_name = window_name if window_name else samples_path.split("\\")[-1]
    cv2.setWindowTitle(window, video_name)
    if skip_empty and not boxes:
        return True
    for i in range(*frames):
        img = cv2.imread(path2img(i, samples_path+r"\images"), cv2.IMREAD_ANYCOLOR)
        if str(i) in map2indices:
            for indicie in map2indices[str(i)]:
                box = boxes[indicie]
                # Get coordinates from your box data
                x = float(box['xtl'])
                y = float(box['ytl'])
                x2 = float(box['xbr'])
                y2 = float(box['ybr'])
                # Convert to integers
                x, y, x2, y2 = map(int, [x, y, x2, y2])
                # Draw rectangle
                cv2.rectangle(img, (x, y), (x2, y2), color, thickness)
        key = cv2.waitKey(2) & 0xFF
        if key == ord("q"):
            return True
        elif key == ord("a"):
            return False
        #samples_path.split("\\")[-1],
        cv2.imshow(window, img)
    return True



In [49]:
cv2.destroyAllWindows()

In [276]:
color = (0, 255, 0)  # Green
thickness = 10
cv2.setWindowTitle("window", "test")
x,y = 1920, 1080
labels = loadLabelsYolo(r"D:\Dataset\FishDetection\Annotation\task_20230718031518_20230718031525_3.mp4_dataset_2024_02_26_06_58_29_cvat for video 1.1\images\frame_000000.txt")
img = cv2.imread(r"D:\Dataset\FishDetection\Annotation\task_20230718031518_20230718031525_3.mp4_dataset_2024_02_26_06_58_29_cvat for video 1.1\images\frame_000000.PNG", cv2.IMREAD_COLOR)
for label in labels:
    start, stop = labelToBox(label, (x,y))
    cv2.rectangle(img, start, stop, color, thickness)
cv2.imshow("window", img)
key = cv2.waitKey(10075)
if key == ord("q"):
    cv2.destroyAllWindows()

In [272]:
cv2.destroyAllWindows()